In [ ]:
import numpy as np
import numba
from numba import jit

In [ ]:
print(numba.__version__)
scale = 1e-3

0.56.4


In [ ]:
########################################################
## Convolution Operation
## Optimized using NP einsum
## Reference: https://ajcr.net/Basic-guide-to-einsum/
########################################################

## Function to get Windows 

def fetch_windows(input: np.ndarray, conv_dim, kernel_size, padding=0):
    # Create a ref to the input
    input_copy = input

    if (padding != 0):
        input_copy = np.pad(input_copy, 
                               pad_width=((0,), (0,), (padding,), (padding,)), mode='constant', constant_values=(0.,))

    batch_str, channel_str, kern_h_str, kern_w_str = input_copy.strides
    # Get the h and w of the convolution output 
    out_h = conv_dim[0]
    out_w = conv_dim[1]

    # Unpack 
    out_batch, out_channels, _, _ = input_copy.shape
    
    return np.lib.stride_tricks.as_strided(
        input_copy,
        (out_batch, out_channels, out_h, out_w, kernel_size, kernel_size),
        (batch_str, channel_str, kern_h_str, kern_w_str, kern_h_str, kern_w_str)
    )

def fetch_raw_windows(in_x, win_dims, strides):
    win = np.lib.stride_tricks.as_strided(in_x, win_dims, strides)
    return win

In [ ]:
###############################################################
## Implementation based on Vectorized CNN 
## Using NP Stride Tricks and EinSum 
## https://jessicastringham.net/2017/12/31/stride-tricks/
## https://ajcr.net/Basic-guide-to-einsum/
###############################################################

class ConvolutionLayer:
    def __init__(self, in_channels, out_channels, kernel_size):
        # self.input_h = in_h
        # self.input_w = in_w
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size

        # Xavier Initialization 
        self._init_weights_bias()
        self.input = None
        self.windows = None
        self.grads = None
        self.id = 0
        # Adam optimizer params
        self.m_dw = 0
        self.m_db = 0
        self.v_dw = 0
        self.v_db = 0 

    def _init_weights_bias(self):
        # self.kernel = np.random.normal(0, 0.001, size=(out_channels, in_channels, kernel_size, kernel_size))
        # self.kernel = np.ones(shape=(out_channels, in_channels, kernel_size, kernel_size))*scale
        ## XAVIER INITIALIZATION
        limit = np.sqrt(2/(self.in_channels+self.out_channels))
        self.kernel = np.random.randn(self.out_channels, self.in_channels, self.kernel_size, self.kernel_size)*limit*scale
        self.bias = np.zeros(self.out_channels)

    def forward(self, x: np.ndarray):
        
        ## Forward Pass
        ## Assume input x is of the form:
        ## x : (B, c_in, h, w)
        ## We want to convolve X and kernel
        ## Create windows from X of the shape of the kernel

        b, c, in_h, in_w = x.shape

        self.output_h = in_h - self.kernel_size + 1
        self.output_w = in_w - self.kernel_size + 1

        windows = fetch_windows(x, (self.output_h, self.output_w), self.kernel_size)

        # b = batch size 
        # n = num of output channels 
        # i = num input channels 
        # h,w = output height, weight dim
        # j, k = kernel dimensions 

        # Output eq
        
        output = np.einsum('bihwjk,nijk->bnhw', windows, self.kernel, optimize=True)
        
        output += self.bias[None, :, None, None]

        # We get an output of size (batch x output channels x h x w) as reqd 
        
        # Output: (B, c_out, output_h, output_w)
        self.input = x
        self.windows = windows

        return output
    
    def backward(self, dout, alpha = 0.001):
        # t1 = time.time()
        # print("Conv back, shape:", dout.shape)
        # Get saved x and windows
        x = self.input
        windows = self.windows

        # print("Conv back, windows shape:", windows.shape)

        # n = num of output channels 
        # i = num input channels 
        # j,k = kernel dimensions 
        # b,c = dimensions of n_windows arr

        # Gradient w.r.t. k
        dk = np.einsum('bihwjk,bnhw->nijk',windows,dout,optimize=True)

        # Gradient w.r.t. bias
        db = np.sum(dout, axis=(0,2,3))

        # Gradient w.r.t. x
        pad = self.kernel_size - 1
        dout_windows = fetch_windows(dout, (x.shape[2], x.shape[3]), self.kernel_size, padding=pad)

        # Flip the kernel 180 degrees 
        flip = np.rot90(self.kernel, 2, axes=(2, 3))
        
        # Gradient w.r.t. input (x)
        # CHECK THIS! 
        dx = np.einsum('bnhwjk,nijk->bihw', dout_windows, flip, optimize=True)

        # print("Grad shape:", dx.shape)

        # dx = np.einsum('nijk,abcijk->ibc',kernel_rot, dout_windows)

        self.grads = (dk,db)

        # Return the gradients
        # To be used by the Adam Optimizer
        # t2 = time.time()
        # print("Time for back pas:", t2-t1)
        return dx

    def update(self, beta1, beta2, epsilon, alpha, t):
        #beta1 : exp decay parameter for 1st moment 
        #beta2 : exp decay parameter for 2nd moment 
        dw = self.grads[0]
        db = self.grads[1]

        # Update 1st moment estimates
        self.m_dw = beta1*self.m_dw + (1-beta1)*dw
        self.m_db = beta1*self.m_db + (1-beta1)*db

        # Update 2nd moment estimates
        self.v_dw = beta2*self.v_dw + (1-beta2)*(dw**2)
        self.v_db = beta2*self.v_db + (1-beta2)*(db**2)

        ## bias correction
        m_dw_corr = self.m_dw/(1-beta1**t)
        m_db_corr = self.m_db/(1-beta1**t)
        v_dw_corr = self.v_dw/(1-beta2**t)
        v_db_corr = self.v_db/(1-beta2**t)

        ## update w and biases
        self.kernel = self.kernel - alpha*(m_dw_corr/(np.sqrt(v_dw_corr)+epsilon))
        self.bias = self.bias - alpha*(m_db_corr/(np.sqrt(v_db_corr)+epsilon))
        return

In [ ]:
#import numpy as np

class FCLayer:
    def __init__(self, nrows, ncols):
        self.nrows = nrows 
        self.ncols = ncols
        self.input = None
        self._init_weights_bias()
        self.grads = None
        self.id = 1
        # Adam optimizer params
        self.m_dw = 0
        self.m_db = 0
        self.v_dw = 0
        self.v_db = 0 

    def _init_weights_bias(self):
        f_in = self.nrows
        f_out = self.ncols
        limit = np.sqrt(2/float(f_in+f_out))

        # He Initialization of weights and bias
        self.weights = np.random.randn(f_in, f_out)*limit*scale
        # self.weights = np.random.randn(-1*limit, limit, size=(f_in, f_out))*scale
        # self.weights = np.random.normal(0.0, limit, size=(f_in, f_out))
        # self.weights = np.random.rand*scale
        # self.weights = np.random.normal(0, 0.001, size=(f_in,f_out))

        # print(self.weights[0])
        self.bias = np.zeros(f_out)

    def forward(self, fc: np.ndarray):
        ## Forward Pass
        ## Assume input x is of the form:``
        ## x : (B x nrows)

        # Calculate the dot product 
        out = np.dot(fc, self.weights) + self.bias

        # Save the input for backprop
        self.input = fc

        return out
    
    def backward(self, dout, alpha = 0.001):

        fc = self.input

        # Get the batch size 
        m = fc.shape[0]

        # Weight gradient
        dw = np.dot(fc.T, dout)

        # Bias gradient 
        db = np.sum(dout, axis = 0)
    
        # Input gradient 
        dx = np.dot(dout, self.weights.T) 

        self.grads = (dw, db)
        
        return dx

    def update(self, beta1, beta2, epsilon, alpha, t):
        #beta1 : exp decay parameter for 1st moment 
        #beta2 : exp decay parameter for 2nd moment 
        dw = self.grads[0]
        db = self.grads[1]

        # Update 1st moment estimates
        self.m_dw = beta1*self.m_dw + (1-beta1)*dw
        self.m_db = beta1*self.m_db + (1-beta1)*db

        # Update 2nd moment estimates
        self.v_dw = beta2*self.v_dw + (1-beta2)*(dw**2)
        self.v_db = beta2*self.v_db + (1-beta2)*(db**2)

        ## bias correction
        m_dw_corr = self.m_dw/(1-beta1**t)
        m_db_corr = self.m_db/(1-beta1**t)
        v_dw_corr = self.v_dw/(1-beta2**t)
        v_db_corr = self.v_db/(1-beta2**t)

        ## update w and biases
        self.weights = self.weights - alpha*(m_dw_corr/(np.sqrt(v_dw_corr)+epsilon))
        self.bias = self.bias - alpha*(m_db_corr/(np.sqrt(v_db_corr)+epsilon))
        return

In [ ]:


class MaxPool:
    def __init__(self, pool_size, stride=1, padding=0):
        self.pool_size = pool_size
        self.stride = stride
        self.padding = padding
        self.input = None
        self.windows = None
        self.indices = None
        self.in_shape = None
        self.out_shape = None
        self.grads = None
        self.id = 2

    def forward(self, x : np.ndarray):
        # Input is of the format (batch, c, w, h)

        b, c, h, w, = x.shape

        # stride = self.stride
        pool = self.pool_size

        # mat_pad = x[:, :, : (h - pool) // stride * stride + pool, : (w - pool) // stride * stride + pool]

        out_h = 1 + ((h - pool)//pool)
        out_w = 1 + ((w - pool)//pool)

        b_str, ch_str, k_h_str, k_w_str = x.strides

        # View is of the form (batch, c, out_w, out_h, pool, pool)
        view = fetch_raw_windows(x, (b, c, out_h, out_w, pool, pool), \
          (x.strides[0], x.strides[1], x.strides[2]*pool, x.strides[3]*pool,x.strides[2], x.strides[3]))

        # Store indices of max elements 
        new_shape = view.shape[:4] + (pool * pool, )
        tmp = view.reshape(new_shape)
        self.indices = np.argmax(view.reshape(new_shape), axis=4)
        out = np.max(tmp, axis=4)

        self.in_shape = x.shape
        self.out_shape = out.shape
       
        return out

    def backward(self, dout : np.ndarray):
        # Input gradients 
        dx = np.zeros(self.in_shape)

        # Get the dimensions of the output
        b, c, out_h, out_w = self.out_shape
        pool = self.pool_size

        # Get a view of the windows 
        view = fetch_raw_windows(dx, (b, c, out_h, out_w, pool, pool), \
          (dx.strides[0], dx.strides[1], dx.strides[2]*pool, dx.strides[3]*pool,dx.strides[2], dx.strides[3]))

        # Set the values in the view 
        for b1 in range(b):
            for c1 in range(c):
                for i in range(out_h):
                    for j in range(out_w):
                        tmp = self.indices[b1, c1, i, j]
                        t1 = (int)(tmp // pool)
                        t2 = (int)(tmp % pool)
                        view[b1, c1, i, j, t1, t2] += dout[b1, c1, i, j]

        return dx

    def update(self, beta1, beta2, epsilon, alpha, t):
        return

In [ ]:
class ReLU:
    def __init__(self):
        self.input = None
        self.grads = None
        self.id = 3
        return

    def forward(self, x : np.ndarray):
        z = np.where(x > 0, x, 0)                          
        self.input = x
        return z

    def backward(self, dout : np.ndarray):
        x = self.input
        # mask = np.ones_like(dout)
        # mask[x<=0] = 0.01
        dx = np.array(dout, copy = True)
  
        dx[x <= 0] = 0
        return dx

    def update(self, beta1, beta2, epsilon, alpha, t):
      return

In [ ]:
class ReShape:
    def __init__(self, in_shape, out_shape):
        self.input = None
        self.in_shape = in_shape
        self.out_shape = out_shape
        self.id=4
        self.grads = None
      
    def forward(self, x: np.ndarray):
        # print(x.shape)
        # print(self.in_shape)
        assert(x.shape == self.in_shape)
        y = np.reshape(x,self.out_shape)
        return y

    def backward(self, dout):
        assert (dout.shape == self.out_shape)
        retval = np.reshape(dout, self.in_shape)
        return retval

    def update(self, beta1, beta2, epsilon, alpha, t):
      return

In [ ]:
def cce_loss(logits, labels):
    # logits = output of the last fc layer 
    temp= np.array(logits, copy=True)
    temp -= np.max(temp, axis=1, keepdims=True)
    softmax_probs = np.exp(temp) / np.sum(np.exp(temp), axis=1, keepdims=True)

    # softmax_probs = np.exp(logits) / np.sum(np.exp(logits), axis=1, keepdims=True)

    n = logits.shape[0]

    # Convert targets to one-hot encoding
    labels_one_hot = np.zeros_like(logits)
    labels_one_hot[np.arange(n), labels] = 1

    # Compute loss
    loss = -(1/n) * np.sum(labels_one_hot * np.log(softmax_probs + 1e-6))
    return loss

In [ ]:
def softmax(logits):
    softmax_probs = np.exp(logits) / np.sum(np.exp(logits), axis=1, keepdims=True)
    return softmax_probs

def grad(logits, labels):
    # Assume targets are one hot encoded
    temp= np.array(logits, copy=True)
    temp -= np.max(temp, axis=1, keepdims=True)
    softmax_probs = np.exp(temp) / np.sum(np.exp(temp), axis=1, keepdims=True)
    
    # softmax_probs = np.exp(logits) / np.sum(np.exp(logits), axis=1, keepdims=True)

    n = logits.shape[0]
    # Convert to one hot encoded 
    targets = np.zeros_like(logits)
    targets[np.arange(n), labels] = 1

    # Compute the softmax probabilities
    # Compute the gradient of the cross-entropy loss w.r.t. the logits
    n = logits.shape[0]
    grad = (softmax_probs - targets) 

    return grad

In [ ]:
from torchvision import datasets, transforms

# Define the transformation to be applied to the dataset
transform = transforms.Compose([transforms.ToTensor()])

# Load the CIFAR10 dataset and apply the defined transformation
train_set = datasets.CIFAR10('./data', train=True, download=True,transform=transform)
val_set = datasets.CIFAR10('./data', train=False, download=True,transform=transform)

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
# Convert the PyTorch dataset to a numpy array
train_data = np.array(train_set.data)
train_labels = np.array(train_set.targets)

val_data = np.array(val_set.data)
val_labels = np.array(val_set.targets)

train_data = np.moveaxis(train_data, -1, 1)
val_data = np.moveaxis(val_data, -1, 1)

In [ ]:
def normalize(inp):
    n_channels = inp.shape[1]
    
    # Calculate the mean and standard deviation of each channel of the batch
    mean = np.mean(inp, axis=(0, 2, 3))
    std = np.std(inp, axis=(0, 2, 3))
    
    # Normalize the batch using the mean and standard deviation
    normalized = (inp - mean.reshape(1, n_channels, 1, 1)) / (std.reshape(1, n_channels, 1, 1)+1e-20)
    
    return normalized

In [ ]:
# train_data = normalize(train_data)

In [ ]:
## Define the layers of the model 

batch_size = 32

Conv1 = ConvolutionLayer(in_channels=3, out_channels=32,kernel_size=3)

MP1 = MaxPool(2)

Conv2 = ConvolutionLayer(in_channels=32,out_channels=64,kernel_size=5)

MP2 = MaxPool(2)

Conv3 = ConvolutionLayer(in_channels=64,out_channels=64,kernel_size=3)

FC1 = FCLayer(64*3*3, 64)
FC2 = FCLayer(64,10)

relu1 = ReLU()
relu2 = ReLU()
relu3 = ReLU()
relu4 = ReLU()
relu5 = ReLU()

RS  = ReShape((batch_size, 64, 3, 3), (batch_size, 3*3*64))

layers = [Conv1, relu1, MP1, Conv2, relu2, MP2, Conv3, relu3, RS, FC1, relu4, FC2]

In [ ]:
## ADAM OPTIMIZER
import time 

# Iteration count 
t = 1

n_epochs = 20
# train_size = 512
train_size = train_data.shape[0]
# train_size = 64

batch_idx = 0

for i in range(n_epochs):
    print("Performing epoch: ", i+1)
    print("---------------------------")
    
    iter = 0
    while(batch_idx < train_size):
        iter += 1
        train_batch = train_data[batch_idx:min(batch_idx+batch_size, train_size)]
        train_batch_labels = train_labels[batch_idx:min(batch_idx+batch_size, train_size)]
        out = train_batch

        # Forward Pass  
        for l in layers:
          out = l.forward(out)
      
        loss = cce_loss(out, train_batch_labels)
        if (iter % 500 == 0):
          print("[ITER {}] Loss:".format(iter), loss)

        # Backward Pass
        inp = grad(out, train_batch_labels)

        for l in reversed(layers):
          # print(inp[0])
          inp = l.backward(inp)
        
        # Now all the gradients are set
        # Update
        for l in layers:
          l.update(0.9,0.999,1e-8,0.001,t)
        
        batch_idx += batch_size
        t += 1

    # Calculate Validation Accuracy 
    # val_idx = 0
    # # val_size = val_data.shape[0]
    # val_size = 64
    # correct = 0
    # total = 0
    # classes = [0 for _ in range(10)]
    # classes_total = [0 for _ in range(10)]

    # while (val_idx < val_size):
    #   val_batch = val_data[val_idx:min(val_idx+batch_size, val_size)]
    #   val_batch_labels = val_labels[val_idx:min(val_idx+batch_size, val_size)]

    #   out = val_batch

    #   # Forward Pass  
    #   for l in layers:
    #     out = l.forward(out)

    #   out = softmax(out)

    #   predictions = np.argmax(out, axis=1)
    #   # print(predictions.shape)
    #   # print(val_batch_labels.shape)
    #   for k in range(len(predictions)):
    #     total+=1 
    #     classes_total[predictions[k]] += 1
    #     if (predictions[k]==val_batch_labels[k]):
    #       correct += 1
    #       classes[predictions[k]] += 1
        
    #   # correct += np.sum(predictions == val_batch_labels)
    #   # total += batch_size 

    #   val_idx += batch_size
    
    # print("Validation Accuracy after Epoch {} is:".format(i), correct * 100 / total)
    # print("Class wise scores:", classes, classes_total)
    batch_idx = 0

In [ ]:
# # Optimizer Pass
# # - - - - - - - - - - - - 
# import time 

# n_epochs = 2000
# batch_idx = 0

# # train_size = 32
# # train_size = 512
# train_size = train_data.shape[0]


# for i in range(n_epochs):
#   print("Performing epoch", i)
#   iter = 0
#   while(batch_idx < train_size):
#     # t1 = time.time()
#     iter += 1
#     # Select Batch 
#     train_batch = train_data[batch_idx:min(batch_idx+batch_size, train_size)]
#     train_batch_labels = train_labels[batch_idx:min(batch_idx+batch_size, train_size)]

#     out = train_batch

#     # Forward Pass 
#     # t1 = time.time()
#     # t1 = time.time()
#     for l in layers:
#       out = l.forward(out)
#     # t2 = time.time()
#     # print("Total time for fwd pas:", t2-t1)
#       # print("OUT:", out[0])
#     # t2 = time.time()
#     # print("Fwd time for one iter:", t2-t1)
#     # print("=================================")
  
#     loss = cce_loss(out, train_batch_labels)
    
#     # print("Curr time is:", t)
#     if (iter % 100 ):
#       print("Iter: ", iter, " batch_idx:", batch_idx)
#       print("Loss is ", loss)
#     # Backward Pass 
#     inp = grad(out, train_batch_labels)
#     # print("============================")
#     # print(inp)
#     # print("============================")
#     # t1 = time.time()
#     # t1 = time.time()
#     for l in reversed(layers):
#       inp = l.backward(inp)
#       # print(inp)
#     # t2 = time.time()
#     # print("Total time for back pas:", t2-t1)
#       # if (iter%100):
#       #  print("Gradient:",inp[0])
#       # print("Back gradient shape:", inp.shape)
#     # t2 = time.time()
#     # print("Back time for one iter:", t2-t1)
#     # time.sleep
#     # Now all the gradients are set
#     # Update weights etc.
#     alpha = 0.001
#     for l in layers:
#       if (l.id == 0):
#         l.kernel -= alpha* l.grads[0]
#         l.bias -= alpha* l.grads[1]
#       elif (l.id == 1):
#         l.weights -= alpha* l.grads[0]
#         l.bias -= alpha*l.grads[1]

#     # Completed, increase idx
#     batch_idx += batch_size
#     # t2 = time.time()
#     # print("Time for one iteration:", t2-t1)

#   print("Completed epoch", i)
#   batch_idx = 0

In [ ]:
# # Define the layers of the model 

# batch_size = 32

# FC1 = FCLayer(64*22*22, 64)
# FC2 = FCLayer(64, 10)

# Conv1 = ConvolutionLayer(3, 32, 3)
# Conv2 = ConvolutionLayer(32, 64, 5)
# Conv3 = ConvolutionLayer(64, 64, 3)

# relu1 = ReLU()
# relu2 = ReLU()
# relu3 = ReLU()
# relu4 = ReLU()
# relu5 = ReLU()
# MP1 = MaxPool(2)
# MP2 = MaxPool(2)
# RS  = ReShape((batch_size, 64, 22, 22), (batch_size, 22*22*64))


# layers = [Conv1, relu1, MP1, Conv2, relu2, MP2, Conv3, relu3, RS, FC1, relu4, FC2]